# TensorFlow 2 quickstart for beginners

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

In [5]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [6]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 17s 1us/step


Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [8]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.40450728,  0.07836689, -1.009729  ,  0.24832651,  0.20193258,
        -0.631343  ,  0.4401044 , -1.0892378 ,  0.70079297,  0.0301787 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [9]:
tf.nn.softmax(predictions).numpy()

array([[0.13726565, 0.09906515, 0.03337082, 0.11741766, 0.11209463,
        0.04871894, 0.14223991, 0.03082029, 0.1846023 , 0.09440458]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [10]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [11]:
loss_fn(y_train[:1], predictions).numpy()

3.0216875

In [12]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [13]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 732us/step - loss: 0.4859 - accuracy: 0.8587
Epoch 2/5
1875/1875 [==============================] - 1s 777us/step - loss: 0.1489 - accuracy: 0.9554
Epoch 3/5
1875/1875 [==============================] - 1s 722us/step - loss: 0.1081 - accuracy: 0.9673
Epoch 4/5
1875/1875 [==============================] - 1s 732us/step - loss: 0.0837 - accuracy: 0.9751
Epoch 5/5
1875/1875 [==============================] - 2s 879us/step - loss: 0.0739 - accuracy: 0.9772


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [14]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0740 - accuracy: 0.9777


[0.07397085428237915, 0.9776999950408936]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [15]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [16]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[6.52170300e-08, 3.98471728e-10, 1.23927648e-05, 1.40055490e-04,
        6.66797659e-11, 2.53495131e-07, 2.57172186e-13, 9.99845862e-01,
        1.83274111e-07, 1.10204780e-06],
       [9.23148434e-07, 6.22306659e-04, 9.99231219e-01, 1.12307789e-04,
        6.82603185e-15, 2.09319223e-05, 5.61311140e-07, 8.97718705e-11,
        1.16879619e-05, 1.25516431e-10],
       [6.11973860e-07, 9.98754740e-01, 1.99380462e-04, 4.78196262e-06,
        4.00794925e-05, 2.14668717e-06, 4.76658897e-05, 7.00107543e-04,
        2.47996970e-04, 2.45003230e-06],
       [9.99916792e-01, 3.59572496e-08, 9.81496669e-06, 9.12917585e-07,
        1.10672282e-07, 9.63855746e-06, 1.45758995e-05, 1.85100616e-05,
        5.00033877e-07, 2.90247899e-05],
       [2.06658824e-05, 1.29934975e-07, 1.37966754e-05, 2.47858065e-06,
        9.83138800e-01, 1.04599840e-05, 9.94424045e-05, 7.86559685e-05,
        8.50642027e-06, 1.66271515e-02]], dtype=float32)>